# Annotation tool to create/correct training labels

In [ ]:
import os
import numpy as np
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts
from skimage.io import imread

from inter_view.dashboards import AnnotationDashBoard

import holoviews as hv
hv.extension('bokeh', width=100)

# config

In [ ]:
basedir = '../../data/2D_corrections'
img_subdir = 'raw'
annot_subdir = 'annot'

data_pattern = '{layer}/{fname}.{ext}'
index = ['layer', 'fname']

channel_config = {img_subdir:{'cmap':'gray'},
                  annot_subdir:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(-2**15,2**15-1), 'slider_limits':(-2**15,2**15-1), 'bitdepth':'int16', 'opacity':0.5}}

def set_inactive_tool(plot, element):
    plot.state.toolbar.active_inspect = None

opts.defaults(opts.Image('channel.{}'.format(img_subdir), frame_width=1500),
              opts.Image('channel.{}'.format(annot_subdir), frame_width=1500, clipping_colors={'min': (0, 0, 0, 0)}, clim=(0,2**16-1), hooks=[set_inactive_tool], tools=['hover'], show_title=False),)

# parse files

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[[img_subdir, annot_subdir]]

df

# interactive dashboard

To start drawing select the 'Freehand Drawing Tool' in the toolbar on the right of the image.

- drawing label:
    - -1: un-annotated (does not contribute to the training loss)
    - 0: background
    - [1...] instance labels. last one in the list is always unused
- on click
    - \-: does nothing
    - pick label (pipette): sets label at the clicked position as drawing label
    - fill label (pot filling): fill the label at the clicked label with the current drawing labels (discontinuous)
- lock
    - background: prevent from drawing over the existing background
    - foreground: prevents from drawing over the existing labels
- save: saves the current annotation. The current annotation is also automatically saved when loading a new image
- discard changes: Cancels all changes made since the last save (i.e. not a granual ctrl+z!)

In [ ]:
db = AnnotationDashBoard(df=df,
                         multi_select_levels=['layer'],
                         channel_config=channel_config,
                         composite_channels=[img_subdir],
                         overlay_channels=[annot_subdir],
                         annot_channel=annot_subdir)

db.panel().servable()